# GNEE Event Embedding
This notebook establishes a training pipeline for our Event Embedding model.

## Installing our libraries and required scripts

In [ ]:
!git clone https://github.com/joaopedromattos/pyGAT
!pip install --quiet spektral

fatal: destination path 'pyGAT' already exists and is not an empty directory.


In [ ]:
!pip install git+https://github.com/rmarcacini/sentence-transformers
!pip install gdown
!gdown https://drive.google.com/uc?id=1NV5t1YhyyOzMF5zAovfbSLdZZLvqrfZ_
!unzip distiluse-base-multilingual-cased.zip -d language_model
from sentence_transformers import SentenceTransformer, LoggingHandler
language_model = SentenceTransformer('distiluse-base-multilingual-cased')

  Cloning https://github.com/rmarcacini/sentence-transformers to /tmp/pip-req-build-q23pz3a0
  Running command git clone -q https://github.com/rmarcacini/sentence-transformers /tmp/pip-req-build-q23pz3a0
  Created wheel for sentence-transformers: filename=sentence_transformers-0.3.6-cp36-none-any.whl size=101875 sha256=26345e03a6bc51ff43257f9a78cd9ca5df349277e131cdfd9cbf83b3ea04b0a4
  Stored in directory: /tmp/pip-ephem-wheel-cache-ms1gqh5o/wheels/88/3c/66/55ee9fb698480d5a5116a8257c15dc363323e4922fb8ad361b
Successfully built sentence-transformers
Downloading...
From: https://drive.google.com/uc?id=1NV5t1YhyyOzMF5zAovfbSLdZZLvqrfZ_
To: /content/distiluse-base-multilingual-cased.zip
504MB [00:03, 163MB/s]
Archive:  distiluse-base-multilingual-cased.zip
replace language_model/0_DistilBERT/added_tokens.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
 extracting: language_model/0_DistilBERT/added_tokens.json  
  inflating: language_model/0_DistilBERT/config.json  
  inflating: language_model/

In [ ]:
import os

os.chdir('./pyGAT')

In [ ]:
import networkx as nx
from tqdm import tqdm
import pandas as pd
from google.colab import auth
from google.cloud import bigquery
from google.cloud import bigquery_storage
import numpy as np
import logging
from sklearn.preprocessing import OneHotEncoder, MultiLabelBinarizer, LabelEncoder
from event_graph_utils import mount_graph, regularization, process_event_dataset_from_networkx

# auth.authenticate_user()
# print('Authenticated')

## 5W1H Graph Events

In [ ]:
!gdown --id 1RF_bIo5ndxPhu9SJw-T8HBcuHyaGQGL0

Downloading...
From: https://drive.google.com/uc?id=1RF_bIo5ndxPhu9SJw-T8HBcuHyaGQGL0
To: /content/pyGAT/datasets.tar.gz
22.7MB [00:00, 139MB/s] 


In [ ]:
!tar -xzvf datasets.tar.gz

datasets_runs/
datasets_runs/run_1_google_news_5w1h_graph_hin.nx
datasets_runs/run_6_40er_5w1h_graph_hin.nx
datasets_runs/run_4_bbc_5w1h_graph_hin.nx
datasets_runs/run_8_gold_standard_5w1h_graph_hin.nx
datasets_runs/run_5_bbc_5w1h_graph_hin.nx
datasets_runs/run_9_google_news_5w1h_graph_hin.nx
datasets_runs/run_5_gold_standard_5w1h_graph_hin.nx
datasets_runs/run_2_bbc_5w1h_graph_hin.nx
datasets_runs/run_9_news_cluster_5w1h_graph_hin.nx
datasets_runs/run_7_40er_5w1h_graph_hin.nx
datasets_runs/run_9_gold_standard_5w1h_graph_hin.nx
datasets_runs/run_8_google_news_5w1h_graph_hin.nx
datasets_runs/run_10_bbc_5w1h_graph_hin.nx
datasets_runs/run_8_news_cluster_5w1h_graph_hin.nx
datasets_runs/run_2_news_cluster_5w1h_graph_hin.nx
datasets_runs/run_8_40er_5w1h_graph_hin.nx
datasets_runs/run_6_bbc_5w1h_graph_hin.nx
datasets_runs/run_4_google_news_5w1h_graph_hin.nx
datasets_runs/run_2_google_news_5w1h_graph_hin.nx
datasets_runs/run_7_gold_standard_5w1h_graph_hin.nx
datasets_runs/run_4_gold_standard_

# Regularization

In [ ]:
import numpy as np
from tqdm.notebook import tqdm
import random


def regularization(G, dim, embedding_feature: str = 'embedding', iterations=15, mi=0.85):

    nodes = []

    # inicializando vetor f para todos os nodes
    for node in G.nodes():
        G.nodes[node]['f'] = np.array([0.0]*dim)
        if embedding_feature in G.nodes[node]:
            G.nodes[node]['f'] = G.nodes[node][embedding_feature]*1.0
        nodes.append(node)

    pbar = tqdm(range(0, iterations))

    for iteration in pbar:
        random.shuffle(nodes)
        energy = 0.0

        # percorrendo cada node
        for node in nodes:
            f_new = np.array([0.0]*dim)
            f_old = np.array(G.nodes[node]['f'])*1.0
            sum_w = 0.0

            # percorrendo vizinhos do onde
            for neighbor in G.neighbors(node):
                w = 1.0
                if 'weight' in G[node][neighbor]:
                    w = G[node][neighbor]['weight']

                w /= np.sqrt(G.degree[neighbor])

                f_new += w*G.nodes[neighbor]['f']

                sum_w += w

            f_new /= sum_w

            G.nodes[node]['f'] = f_new*1.0

            if embedding_feature in G.nodes[node]:
                G.nodes[node]['f'] = G.nodes[node][embedding_feature] * \
                    mi + G.nodes[node]['f']*(1.0-mi)

            energy += np.linalg.norm(f_new-f_old)

        iteration += 1
        message = 'Iteration '+str(iteration)+' | Energy = '+str(energy)
        pbar.set_description(message)

    return G

In [ ]:
import numpy as np
import networkx as nx
import random
from tqdm.notebook import tqdm
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
import logging
from sklearn.preprocessing import OneHotEncoder, MultiLabelBinarizer, LabelEncoder
from sklearn.model_selection import train_test_split


def process_event_dataset_from_networkx(G, features_attr="f"):
    """
    Builds an event graph dataset used in GAT model
    Parameters:
        G -> Graph representation of the event network (Networkx graph)
        df_labels -> user labeled data
        features_att -> Feature attribute of each node (str)
        random_state -> A random seed to train_test_split
    Returns:
        adj -> Sparse and symmetric adjacency matrix of our graph.
        features -> A NumPy matrix with our graph features.
        idx_train -> A NumPy array with the indexes of the training nodes.
        idx_val -> A NumPy array with the indexes of the validation nodes.
        idx_test -> A NumPy array with the indexes of the test nodes.
    """

    num_nodes = len(G.nodes)


    

    # validation_split_percentage = val_split / (1 - train_split)

    # df_val, df_test = train_test_split(
    #     df_test_and_val, train_size=validation_split_percentage, random_state=random_state)

    # Organizing our feature matrix...
    # feature_matrix = np.array([ G.nodes[i]['embedding'] if 'embedding' in G.nodes[i].keys() else G.nodes[i][features_attr] for i in G.nodes()])
    #features = np.array([G.nodes[i][features_attr] for i in G.nodes()])
    L_features = []
    L_train = []
    L_test = []
    L_labels = []
    label_codes = {}
    for node in G.nodes():
      L_features.append( (G.nodes[node]['id'], G.nodes[node]['f']) )
      if 'train' in G.nodes[node]: L_train.append(G.nodes[node]['id'])
      if 'test' in G.nodes[node]: L_test.append(G.nodes[node]['id'])
      if 'label' in G.nodes[node]:
        if G.nodes[node]['label'] not in label_codes: label_codes[G.nodes[node]['label']] = len(label_codes) 
        L_labels.append( [G.nodes[node]['id'],G.nodes[node]['label'],label_codes[G.nodes[node]['label']]] )
    df_features = pd.DataFrame(L_features)
    df_features.columns = ['node_id','embedding']
    features = np.array(df_features.sort_values(by=['node_id'])['embedding'].to_list())

    idx_train = L_train
    idx_test = L_test
    labels = [-1]*num_nodes
    df_labels = pd.DataFrame(L_labels)
    df_labels.columns = ['event_id','label','label_code']
    for index,row in df_labels.iterrows():
      labels[row['event_id']] = row['label_code']

    adj = nx.adjacency_matrix(G)

    return adj, features, labels, idx_train, idx_test, df_labels

## Training

In [ ]:
from __future__ import division
from __future__ import print_function

import os
import glob
import time
import random
import argparse
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
import scipy.sparse as sp


from models import GAT, SpGAT


def accuracy(output, labels):
    preds = output.max(1)[1].type_as(labels)
    correct = preds.eq(labels).double()
    correct = correct.sum()
    return correct / len(labels)


def normalize_adj(mx):
    """Row-normalize sparse matrix"""
    rowsum = np.array(mx.sum(1))
    r_inv_sqrt = np.power(rowsum, -0.5).flatten()
    r_inv_sqrt[np.isinf(r_inv_sqrt)] = 0.
    r_mat_inv_sqrt = sp.diags(r_inv_sqrt)
    return mx.dot(r_mat_inv_sqrt).transpose().dot(r_mat_inv_sqrt)

class Namespace(object):
    def __init__(self, adict):
        self.__dict__.update(adict)

class GAT_wrapper():
    def __init__(self, args={"alpha": 0.2, "cuda": True, "dropout": 0.6, "epochs": 10, "fastmode": False, "hidden": 8, "lr": 0.005, "nb_heads": 8, "no_cuda": False, "patience": 100, "seed": 72, "sparse": False, "weight_decay": 0.0005}):

        if (type(args) == dict):
            args = Namespace(args)

        self.args = args

        self.model = None

        self.loss_test = 0.0
        self.acc_test = 0.0

        self.adj = None
        self.features = None
        self.labels = None
        self.idx_train = None
        self.idx_val = None
        self.idx_test = None

    def compute_test(self):
        self.model.eval()
        output = self.model(self.features, self.adj)
        loss_test = F.nll_loss(
            output[self.idx_test], self.labels[self.idx_test])
        acc_test = accuracy(output[self.idx_test], self.labels[self.idx_test])
        print("Test set results:",
              "loss= {:.4f}".format(loss_test.item()),
              "accuracy= {:.4f}".format(acc_test.item()))

        self.loss_test = loss_test
        self.acc_test = acc_test

        return loss_test, acc_test, output[self.idx_test].max(1)[1]

    def train_pipeline(self, adj, features, labels, idx_train, idx_val, idx_test, *args):

        adj = normalize_adj(adj + sp.eye(adj.shape[0]))

        if (sp.issparse(adj)):
            adj = adj.todense()

        if (sp.issparse(features)):
            features = features.todense()

        # With networkx, we no longer need to convert from one-hot encoding...
        #labels = np.where(labels)[1]

        adj = torch.FloatTensor(adj)
        features = torch.FloatTensor(features)
        labels = torch.LongTensor(labels)
        idx_train = torch.LongTensor(idx_train)
        idx_val = torch.LongTensor(idx_val)
        idx_test = torch.LongTensor(idx_test)

        random.seed(self.args.seed)
        np.random.seed(self.args.seed)
        torch.manual_seed(self.args.seed)
        if self.args.cuda:
            torch.cuda.manual_seed(self.args.seed)

        # Load data
        # adj, features, labels, idx_train, idx_val, idx_test = new_load_data(
        #     *args, custom_function=custom_function, function=function)

        # Model and optimizer
        if self.args.sparse:
            model = SpGAT(nfeat=features.shape[1],
                          nhid=self.args.hidden,
                          nclass=int(labels.max()) + 1,
                          dropout=self.args.dropout,
                          nheads=self.args.nb_heads,
                          alpha=self.args.alpha)
        else:
            model = GAT(nfeat=features.shape[1],
                        nhid=self.args.hidden,
                        nclass=int(labels.max()) + 1,
                        dropout=self.args.dropout,
                        nheads=self.args.nb_heads,
                        alpha=self.args.alpha)
        optimizer = optim.Adam(model.parameters(),
                               lr=self.args.lr,
                               weight_decay=self.args.weight_decay)

        if self.args.cuda:
            model.cuda()
            features = features.cuda()
            adj = adj.cuda()
            labels = labels.cuda()
            idx_train = idx_train.cuda()
            idx_val = idx_val.cuda()
            idx_test = idx_test.cuda()

        features, adj, labels = Variable(
            features), Variable(adj), Variable(labels)

        # TODO: Test if these lines could be written below line 41.
        self.adj = adj
        self.features = features
        self.labels = labels
        self.idx_train = idx_train
        self.idx_val = idx_val
        self.idx_test = idx_test

        def train(epoch):
            t = time.time()
            model.train()
            optimizer.zero_grad()
            output = model(features, adj)
            loss_train = F.nll_loss(output[idx_train], labels[idx_train])
            acc_train = accuracy(output[idx_train], labels[idx_train])
            loss_train.backward()
            optimizer.step()

            if not self.args.fastmode:
                # Evaluate validation set performance separately,
                # deactivates dropout during validation run.
                model.eval()
                output = model(features, adj)

            loss_val = F.nll_loss(output[idx_val], labels[idx_val])
            acc_val = accuracy(output[idx_val], labels[idx_val])
            print('Epoch: {:04d}'.format(epoch+1),
                  'loss_train: {:.4f}'.format(loss_train.data.item()),
                  'acc_train: {:.4f}'.format(acc_train.data.item()),
                  'loss_val: {:.4f}'.format(loss_val.data.item()),
                  'acc_val: {:.4f}'.format(acc_val.data.item()),
                  'time: {:.4f}s'.format(time.time() - t))

            return loss_val.data.item()

        # Train model
        t_total = time.time()
        loss_values = []
        bad_counter = 0
        best = self.args.epochs + 1
        best_epoch = 0
        for epoch in range(self.args.epochs):
            loss_values.append(train(epoch))

            torch.save(model.state_dict(), '{}.pkl'.format(epoch))
            if loss_values[-1] < best:
                best = loss_values[-1]
                best_epoch = epoch
                bad_counter = 0
            else:
                bad_counter += 1

            if bad_counter == self.args.patience:
                break

            files = glob.glob('*.pkl')
            for file in files:
                epoch_nb = int(file.split('.')[0])
                if epoch_nb < best_epoch:
                    os.remove(file)

        files = glob.glob('*.pkl')
        for file in files:
            epoch_nb = int(file.split('.')[0])
            if epoch_nb > best_epoch:
                os.remove(file)

        print("Optimization Finished!")
        print("Total time elapsed: {:.4f}s".format(time.time() - t_total))

        # Restore best model
        print('Loading {}th epoch'.format(best_epoch))
        model.load_state_dict(torch.load('{}.pkl'.format(best_epoch)))

        self.model = model

        return model



In [ ]:
from os import listdir
from os.path import isfile, join
path_datasets = 'datasets_runs/'
network_files = [f for f in listdir(path_datasets) if isfile(join(path_datasets, f))]
print(network_files)

['run_1_google_news_5w1h_graph_hin.nx', 'run_6_40er_5w1h_graph_hin.nx', 'run_4_bbc_5w1h_graph_hin.nx', 'run_8_gold_standard_5w1h_graph_hin.nx', 'run_5_bbc_5w1h_graph_hin.nx', 'run_9_google_news_5w1h_graph_hin.nx', 'run_5_gold_standard_5w1h_graph_hin.nx', 'run_2_bbc_5w1h_graph_hin.nx', 'run_9_news_cluster_5w1h_graph_hin.nx', 'run_7_40er_5w1h_graph_hin.nx', 'run_9_gold_standard_5w1h_graph_hin.nx', 'run_8_google_news_5w1h_graph_hin.nx', 'run_10_bbc_5w1h_graph_hin.nx', 'run_8_news_cluster_5w1h_graph_hin.nx', 'run_2_news_cluster_5w1h_graph_hin.nx', 'run_8_40er_5w1h_graph_hin.nx', 'run_6_bbc_5w1h_graph_hin.nx', 'run_4_google_news_5w1h_graph_hin.nx', 'run_2_google_news_5w1h_graph_hin.nx', 'run_7_gold_standard_5w1h_graph_hin.nx', 'run_4_gold_standard_5w1h_graph_hin.nx', 'run_5_40er_5w1h_graph_hin.nx', 'run_3_gold_standard_5w1h_graph_hin.nx', 'run_4_40er_5w1h_graph_hin.nx', 'run_5_google_news_5w1h_graph_hin.nx', 'run_10_news_cluster_5w1h_graph_hin.nx', 'run_10_40er_5w1h_graph_hin.nx', 'run_9_40

In [ ]:
!pwd

/content/pyGAT


In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from tqdm.notebook import tqdm

experimental_results = []

for network_file in tqdm(network_files):

  if 'news_cluster_5w1h_graph_hin.nx' in network_file: continue # usando toda a RAM as vezes???
  print('Networkfile',network_file)
  

  G = nx.read_gpickle(path_datasets+network_file)

  regularization(G, 512, embedding_feature='features')
  adj, features, labels, idx_train, idx_test, df_labels = process_event_dataset_from_networkx(G)
  print(adj.shape,features.shape,len(idx_train),len(idx_test))
  gat = GAT_wrapper({"alpha": 0.2, "cuda": False, "dropout": 0.5, "epochs": 20, "fastmode": False, "hidden": 8, "lr": 0.005, "nb_heads": 8, "no_cuda": False, "patience": 100, "seed": 72, "sparse": False, "weight_decay": 0.0005})
  gat.train_pipeline(adj, features, labels, idx_train, idx_train, idx_test)
  loss, acc, output = gat.compute_test()
  y_pred = output.numpy()
  y_true = []
  for event_id in idx_test:
    for node in G.nodes():
      if ':event' in node:
        if G.nodes[node]['id'] == event_id:
          y_true.append(df_labels[df_labels.event_id==event_id].label_code.values[0])

  f1_macro = f1_score(y_true, y_pred, average='macro')
  acc = accuracy_score(y_true, y_pred)

  print('--->' ,network_file,'f1_macro',f1_macro,'acc',acc)
  experimental_results.append((network_file,'f1_macro',f1_macro,'acc',acc,y_true,y_pred))
  del gat
  del adj
  del features
  del G

Networkfile run_1_google_news_5w1h_graph_hin.nx



(227, 227) (227, 512) 7 25
Epoch: 0001 loss_train: 2.0304 acc_train: 0.0000 loss_val: 1.8262 acc_val: 0.5714 time: 0.2153s
Epoch: 0002 loss_train: 1.9033 acc_train: 0.2857 loss_val: 1.6255 acc_val: 0.8571 time: 0.1324s
Epoch: 0003 loss_train: 1.6974 acc_train: 0.5714 loss_val: 1.4399 acc_val: 1.0000 time: 0.1432s
Epoch: 0004 loss_train: 1.4313 acc_train: 1.0000 loss_val: 1.2700 acc_val: 1.0000 time: 0.1391s
Epoch: 0005 loss_train: 1.3770 acc_train: 1.0000 loss_val: 1.1192 acc_val: 1.0000 time: 0.1372s
Epoch: 0006 loss_train: 1.2877 acc_train: 1.0000 loss_val: 0.9832 acc_val: 1.0000 time: 0.1351s
Epoch: 0007 loss_train: 0.9510 acc_train: 1.0000 loss_val: 0.8558 acc_val: 1.0000 time: 0.1369s
Epoch: 0008 loss_train: 0.8510 acc_train: 1.0000 loss_val: 0.7407 acc_val: 1.0000 time: 0.1371s
Epoch: 0009 loss_train: 0.7358 acc_train: 1.0000 loss_val: 0.6380 acc_val: 1.0000 time: 0.1357s
Epoch: 0010 loss_train: 0.4782 acc_train: 1.0000 loss_val: 0.5456 acc_val: 1.0000 time: 0.1402s
Epoch: 0011 


(249, 249) (249, 512) 8 32
Epoch: 0001 loss_train: 1.0589 acc_train: 0.5000 loss_val: 0.8871 acc_val: 0.8750 time: 0.1742s
Epoch: 0002 loss_train: 0.8413 acc_train: 0.6250 loss_val: 0.7456 acc_val: 0.8750 time: 0.1522s
Epoch: 0003 loss_train: 0.7677 acc_train: 0.8750 loss_val: 0.6309 acc_val: 0.8750 time: 0.1557s
Epoch: 0004 loss_train: 0.6845 acc_train: 0.8750 loss_val: 0.5393 acc_val: 0.8750 time: 0.1514s
Epoch: 0005 loss_train: 0.5172 acc_train: 0.8750 loss_val: 0.4659 acc_val: 0.8750 time: 0.1499s
Epoch: 0006 loss_train: 0.4748 acc_train: 0.8750 loss_val: 0.4082 acc_val: 0.8750 time: 0.1500s
Epoch: 0007 loss_train: 0.4263 acc_train: 0.8750 loss_val: 0.3561 acc_val: 0.8750 time: 0.1572s
Epoch: 0008 loss_train: 0.3458 acc_train: 0.8750 loss_val: 0.3062 acc_val: 0.8750 time: 0.1558s
Epoch: 0009 loss_train: 0.3536 acc_train: 0.8750 loss_val: 0.2581 acc_val: 1.0000 time: 0.1542s
Epoch: 0010 loss_train: 0.3201 acc_train: 0.8750 loss_val: 0.2162 acc_val: 1.0000 time: 0.1538s
Epoch: 0011 


(392, 392) (392, 512) 11 44
Epoch: 0001 loss_train: 1.5693 acc_train: 0.1818 loss_val: 1.4092 acc_val: 0.6364 time: 0.3858s
Epoch: 0002 loss_train: 1.3964 acc_train: 0.5455 loss_val: 1.2574 acc_val: 0.8182 time: 0.3689s
Epoch: 0003 loss_train: 1.2667 acc_train: 0.7273 loss_val: 1.1140 acc_val: 1.0000 time: 0.3705s
Epoch: 0004 loss_train: 1.0144 acc_train: 0.9091 loss_val: 0.9827 acc_val: 1.0000 time: 0.3801s
Epoch: 0005 loss_train: 1.0544 acc_train: 1.0000 loss_val: 0.8652 acc_val: 1.0000 time: 0.3667s
Epoch: 0006 loss_train: 0.8229 acc_train: 1.0000 loss_val: 0.7588 acc_val: 1.0000 time: 0.3708s
Epoch: 0007 loss_train: 0.8366 acc_train: 1.0000 loss_val: 0.6647 acc_val: 1.0000 time: 0.3676s
Epoch: 0008 loss_train: 0.5940 acc_train: 1.0000 loss_val: 0.5800 acc_val: 1.0000 time: 0.3629s
Epoch: 0009 loss_train: 0.6747 acc_train: 1.0000 loss_val: 0.5053 acc_val: 1.0000 time: 0.3799s
Epoch: 0010 loss_train: 0.5745 acc_train: 1.0000 loss_val: 0.4405 acc_val: 1.0000 time: 0.3670s
Epoch: 0011


(579, 579) (579, 512) 20 76
Epoch: 0001 loss_train: 2.6307 acc_train: 0.0000 loss_val: 2.3992 acc_val: 0.5000 time: 0.8237s
Epoch: 0002 loss_train: 2.4198 acc_train: 0.4000 loss_val: 2.2197 acc_val: 0.7000 time: 0.7964s
Epoch: 0003 loss_train: 2.2602 acc_train: 0.5000 loss_val: 2.0506 acc_val: 0.7500 time: 0.7933s
Epoch: 0004 loss_train: 2.1604 acc_train: 0.6000 loss_val: 1.8930 acc_val: 0.7500 time: 0.8128s
Epoch: 0005 loss_train: 1.8927 acc_train: 0.8000 loss_val: 1.7419 acc_val: 0.8500 time: 0.7932s
Epoch: 0006 loss_train: 1.8001 acc_train: 0.8500 loss_val: 1.6000 acc_val: 0.8500 time: 0.7901s
Epoch: 0007 loss_train: 1.6835 acc_train: 0.8000 loss_val: 1.4673 acc_val: 0.8500 time: 0.7886s
Epoch: 0008 loss_train: 1.4789 acc_train: 0.8500 loss_val: 1.3436 acc_val: 0.8500 time: 0.7910s
Epoch: 0009 loss_train: 1.6575 acc_train: 0.8500 loss_val: 1.2296 acc_val: 0.8500 time: 0.7863s
Epoch: 0010 loss_train: 1.2907 acc_train: 0.8000 loss_val: 1.1230 acc_val: 0.9000 time: 0.7958s
Epoch: 0011

...


(392, 392) (392, 512) 11 44
Epoch: 0001 loss_train: 1.6593 acc_train: 0.2727 loss_val: 1.4293 acc_val: 0.8182 time: 0.3591s
Epoch: 0002 loss_train: 1.4760 acc_train: 0.8182 loss_val: 1.2727 acc_val: 1.0000 time: 0.3589s
Epoch: 0003 loss_train: 1.3252 acc_train: 0.9091 loss_val: 1.1283 acc_val: 1.0000 time: 0.3659s
Epoch: 0004 loss_train: 1.0891 acc_train: 1.0000 loss_val: 0.9949 acc_val: 1.0000 time: 0.3590s
Epoch: 0005 loss_train: 1.0269 acc_train: 1.0000 loss_val: 0.8723 acc_val: 1.0000 time: 0.3615s
Epoch: 0006 loss_train: 0.9384 acc_train: 1.0000 loss_val: 0.7615 acc_val: 1.0000 time: 0.3587s
Epoch: 0007 loss_train: 0.8201 acc_train: 1.0000 loss_val: 0.6633 acc_val: 1.0000 time: 0.3573s
Epoch: 0008 loss_train: 0.6755 acc_train: 1.0000 loss_val: 0.5759 acc_val: 1.0000 time: 0.3679s
Epoch: 0009 loss_train: 0.6161 acc_train: 1.0000 loss_val: 0.4986 acc_val: 1.0000 time: 0.3573s
Epoch: 0010 loss_train: 0.5397 acc_train: 1.0000 loss_val: 0.4308 acc_val: 1.0000 time: 0.3568s
Epoch: 0011


(249, 249) (249, 512) 8 32
Epoch: 0001 loss_train: 1.0816 acc_train: 0.5000 loss_val: 0.8192 acc_val: 1.0000 time: 0.1601s
Epoch: 0002 loss_train: 0.8443 acc_train: 1.0000 loss_val: 0.6522 acc_val: 1.0000 time: 0.1514s
Epoch: 0003 loss_train: 0.6403 acc_train: 1.0000 loss_val: 0.5218 acc_val: 1.0000 time: 0.1516s
Epoch: 0004 loss_train: 0.5227 acc_train: 1.0000 loss_val: 0.4176 acc_val: 1.0000 time: 0.1501s
Epoch: 0005 loss_train: 0.3525 acc_train: 1.0000 loss_val: 0.3348 acc_val: 1.0000 time: 0.1515s
Epoch: 0006 loss_train: 0.3876 acc_train: 1.0000 loss_val: 0.2702 acc_val: 1.0000 time: 0.1596s
Epoch: 0007 loss_train: 0.2493 acc_train: 1.0000 loss_val: 0.2190 acc_val: 1.0000 time: 0.1474s
Epoch: 0008 loss_train: 0.2948 acc_train: 1.0000 loss_val: 0.1789 acc_val: 1.0000 time: 0.1535s
Epoch: 0009 loss_train: 0.2158 acc_train: 1.0000 loss_val: 0.1474 acc_val: 1.0000 time: 0.1500s
Epoch: 0010 loss_train: 0.2596 acc_train: 1.0000 loss_val: 0.1230 acc_val: 1.0000 time: 0.1585s
Epoch: 0011 


(392, 392) (392, 512) 11 44
Epoch: 0001 loss_train: 1.7074 acc_train: 0.2727 loss_val: 1.4944 acc_val: 0.7273 time: 0.3575s
Epoch: 0002 loss_train: 1.5133 acc_train: 0.3636 loss_val: 1.3505 acc_val: 0.8182 time: 0.3580s
Epoch: 0003 loss_train: 1.3634 acc_train: 0.7273 loss_val: 1.2156 acc_val: 0.8182 time: 0.3675s
Epoch: 0004 loss_train: 1.1908 acc_train: 0.8182 loss_val: 1.0887 acc_val: 1.0000 time: 0.3534s
Epoch: 0005 loss_train: 1.0608 acc_train: 1.0000 loss_val: 0.9713 acc_val: 1.0000 time: 0.3551s
Epoch: 0006 loss_train: 0.9793 acc_train: 1.0000 loss_val: 0.8641 acc_val: 1.0000 time: 0.3575s
Epoch: 0007 loss_train: 0.8824 acc_train: 1.0000 loss_val: 0.7691 acc_val: 1.0000 time: 0.3585s
Epoch: 0008 loss_train: 0.6629 acc_train: 1.0000 loss_val: 0.6823 acc_val: 1.0000 time: 0.3515s
Epoch: 0009 loss_train: 0.7614 acc_train: 1.0000 loss_val: 0.6048 acc_val: 1.0000 time: 0.3563s
Epoch: 0010 loss_train: 0.9140 acc_train: 0.8182 loss_val: 0.5376 acc_val: 1.0000 time: 0.3711s
Epoch: 0011


(249, 249) (249, 512) 8 32
Epoch: 0001 loss_train: 1.1056 acc_train: 0.2500 loss_val: 0.8812 acc_val: 0.8750 time: 0.1636s
Epoch: 0002 loss_train: 0.9367 acc_train: 0.8750 loss_val: 0.7198 acc_val: 1.0000 time: 0.1543s
Epoch: 0003 loss_train: 0.7221 acc_train: 0.8750 loss_val: 0.5924 acc_val: 1.0000 time: 0.1526s
Epoch: 0004 loss_train: 0.5983 acc_train: 1.0000 loss_val: 0.4897 acc_val: 1.0000 time: 0.1535s
Epoch: 0005 loss_train: 0.4494 acc_train: 1.0000 loss_val: 0.4058 acc_val: 1.0000 time: 0.1551s
Epoch: 0006 loss_train: 0.3136 acc_train: 1.0000 loss_val: 0.3357 acc_val: 1.0000 time: 0.1559s
Epoch: 0007 loss_train: 0.2429 acc_train: 1.0000 loss_val: 0.2789 acc_val: 1.0000 time: 0.1536s
Epoch: 0008 loss_train: 0.3799 acc_train: 1.0000 loss_val: 0.2323 acc_val: 1.0000 time: 0.1579s
Epoch: 0009 loss_train: 0.4393 acc_train: 1.0000 loss_val: 0.1938 acc_val: 1.0000 time: 0.1509s
Epoch: 0010 loss_train: 0.3991 acc_train: 1.0000 loss_val: 0.1621 acc_val: 1.0000 time: 0.1642s
Epoch: 0011 


(249, 249) (249, 512) 8 32
Epoch: 0001 loss_train: 1.0945 acc_train: 0.1250 loss_val: 0.8817 acc_val: 0.6250 time: 0.1545s
Epoch: 0002 loss_train: 0.9633 acc_train: 0.6250 loss_val: 0.7324 acc_val: 0.8750 time: 0.1524s
Epoch: 0003 loss_train: 0.7440 acc_train: 0.7500 loss_val: 0.6117 acc_val: 0.8750 time: 0.1553s
Epoch: 0004 loss_train: 0.6839 acc_train: 0.8750 loss_val: 0.5129 acc_val: 1.0000 time: 0.1541s
Epoch: 0005 loss_train: 0.4575 acc_train: 0.8750 loss_val: 0.4258 acc_val: 1.0000 time: 0.1485s
Epoch: 0006 loss_train: 0.4037 acc_train: 1.0000 loss_val: 0.3529 acc_val: 1.0000 time: 0.1505s
Epoch: 0007 loss_train: 0.3594 acc_train: 0.8750 loss_val: 0.2938 acc_val: 1.0000 time: 0.1443s
Epoch: 0008 loss_train: 0.2779 acc_train: 1.0000 loss_val: 0.2448 acc_val: 1.0000 time: 0.1554s
Epoch: 0009 loss_train: 0.3686 acc_train: 0.8750 loss_val: 0.2052 acc_val: 1.0000 time: 0.1537s
Epoch: 0010 loss_train: 0.1952 acc_train: 1.0000 loss_val: 0.1723 acc_val: 1.0000 time: 0.1564s
Epoch: 0011 

In [ ]:
df_results = pd.DataFrame(experimental_results)
df_results

,0,1,2,3,4,5,6
0,run_1_google_news_5w1h_graph_hin.nx,f1_macro,0.941392,acc,0.960000,"[1, 0, 4, 2, 2, 5, 0, 2, 5, 6, 5, 4, 2, 1, 3, ...","[1, 0, 4, 2, 2, 5, 0, 2, 5, 6, 5, 4, 2, 2, 3, ..."
1,run_6_40er_5w1h_graph_hin.nx,f1_macro,0.851648,acc,0.812500,"[0, 1, 1, 0, 0, 2, 1, 0, 1, 0, 1, 0, 1, 1, 2, ...","[0, 1, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 1, 1, 2, ..."
2,run_4_bbc_5w1h_graph_hin.nx,f1_macro,0.584537,acc,0.590909,"[1, 2, 3, 1, 2, 0, 3, 3, 0, 4, 3, 0, 3, 2, 2, ...","[0, 2, 2, 1, 2, 0, 3, 0, 0, 2, 3, 0, 0, 2, 2, ..."
3,run_8_gold_standard_5w1h_graph_hin.nx,f1_macro,0.612381,acc,0.750000,"[0, 1, 3, 4, 5, 0, 1, 6, 3, 0, 5, 4, 6, 1, 8, ...","[0, 1, 3, 9, 9, 0, 1, 6, 3, 0, 5, 9, 6, 1, 8, ..."
4,run_5_bbc_5w1h_graph_hin.nx,f1_macro,0.779684,acc,0.772727,"[1, 2, 3, 1, 2, 0, 3, 0, 4, 3, 0, 2, 2, 4, 0, ...","[1, 1, 3, 1, 2, 0, 3, 0, 2, 3, 0, 2, 2, 0, 0, ..."
5,run_9_google_news_5w1h_graph_hin.nx,f1_macro,1.000000,acc,1.000000,"[0, 2, 0, 4, 2, 2, 5, 0, 5, 5, 6, 5, 4, 1, 2, ...","[0, 2, 0, 4, 2, 2, 5, 0, 5, 5, 6, 5, 4, 1, 2, ..."
6,run_5_gold_standard_5w1h_graph_hin.nx,f1_macro,0.705370,acc,0.815789,"[0, 1, 2, 3, 4, 0, 1, 6, 0, 1, 5, 7, 0, 4, 6, ...","[0, 1, 2, 3, 8, 0, 1, 6, 0, 1, 5, 1, 0, 5, 6, ..."
7,run_2_bbc_5w1h_graph_hin.nx,f1_macro,0.562032,acc,0.568182,"[1, 2, 2, 3, 1, 2, 3, 0, 4, 3, 0, 3, 2, 2, 4, ...","[0, 1, 2, 2, 0, 3, 0, 0, 2, 3, 0, 2, 2, 2, 0, ..."
8,run_7_40er_5w1h_graph_hin.nx,f1_macro,0.819264,acc,0.906250,"[0, 0, 1, 0, 0, 1, 2, 0, 1, 0, 1, 1, 0, 1, 1, ...","[0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, ..."
9,run_9_gold_standard_5w1h_graph_hin.nx,f1_macro,0.698911,acc,0.815789,"[0, 0, 2, 3, 4, 5, 1, 0, 1, 6, 3, 0, 1, 5, 7, ...","[0, 0, 2, 3, 8, 5, 1, 0, 1, 6, 3, 0, 1, 5, 1, ..."


In [ ]:
df_results.to_excel('gat_results_r1.xls')